In [1]:
import numpy as np
import operator
import statistics as s

class Atom:
    """ A Class for storing atom information """

    def __init__(self):
        """ Initialise the class """
        self.id = 0                                              # id of the atom
        self.type = 0                                            # type of the atom
        self.L = np.array([0.0,0.0,0.0],dtype=np.float64)        # size of simulation box
        self.x = np.array([0.0,0.0,0.0],dtype=np.float64)        # position of the atom
        self.image = np.array([0,0,0],dtype=np.int32)            # image flags for atoms
        self.x_unwrap = np.array([0.0,0.0,0.0],dtype=np.float64) # position of the atom - unwrapped coords
        self.unwrap_flag = False

    
    def sep(self, atom2):
        """Takes in self and atom2 and finds separation between them taking into account periodic BCs"""

        dx = self.x[0] - atom2.x[0]
        dy = self.x[1] - atom2.x[1]
        dz = self.x[2] - atom2.x[2]

        if dx > self.L[0] / 2:
            dx = self.L[0] - dx
        if dy > self.L[1] / 2:
            dy = self.L[1] - dy
        if dz > self.L[2] / 2:
            dz = self.L[2] - dz

        return np.sqrt(dx**2 + dy**2 + dz**2)
    
    def sep_2(self, atom2):
        """ Takes in self and atom2 and finds their separation SQUARED WITHOUT taking into account periodic BCs """

        dx = self.x[0] - atom2.x[0]
        dy = self.x[1] - atom2.x[1]
        dz = self.x[2] - atom2.x[2]

        """
        if dx > self.half_L[0]:
            dx = self.L[0] - dx
        if dy > self.half_L[1]:
            dy = self.L[1] - dy
        if dz > self.half_L[2]:
            dz = self.L[2] - dz
        """

        return dx**2 + dy**2 + dz**2

    def minus(self,B):
        """ Subtract B.x vector from self.x vector """
        
        AminusB = np.array([0.0,0.0,0.0],dtype=np.float64)
        for i in range(3):
            AminusB[i] = self.x[i] - B.x[i]
            
        return AminusB

    def xdot(self,B):
        """ Find dot product of position x of this Atom and Atom B """
        
        AdotB = np.array([0.0,0.0,0.0],dtype=np.float64) # TO DO : find AdotB ??
        
        return AdotB

    # confused as to what this function does :/ - isn't unwrap_flag always False?
    def unwrap(self):
        """ Unwraps the coordinates for periodic box to generate x_unwrap array """
        
        if not self.unwrap_flag:   # first check it has not already been done
            for j in range(3):
                self.x_unwrap[j] = self.x[j] + self.image[j]*self.L[j] # unwrap
            unwrap_flag = True

In [2]:
def readframe(infile, N):
    """ Read a single frame of N atoms from a dump file 
        Expects coordinates to be in range -L/2 to L/2
        DOES NOT Unwrap corrdinates for periodic box """

    atoms = [Atom() for i in range(N)]
    L = np.array([0.0,0.0,0.0],dtype=np.float64)

    # read in the 9 header lines of the dump file
    # get box size
    for i in range(9):
        line = infile.readline()
        if i==1:  ## second line of frame is timestep
            timestep = np.int32(line)
        if i==5 or i==6 or i==7:   # 6th-8th lines are box size in x,y,z dimensions
            # get the box size
            line = line.split()
            L[i-5]=np.float64(line[1]) - np.float64(line[0]);

    # now read the atoms, putting them at the correct index (index=id-1)
    for i in range(N):
        line = infile.readline()
        line = line.split()
        index = int(line[0])-1  # LAMMPS atom ids start from 1, python index starts from 0
        atoms[index].id = int(line[0])
        atoms[index].type = int(line[1])
        atoms[index].L = L
        for j in range(3):
            atoms[index].x[j] = np.float64(line[j+2])
        for j in range(3):
            atoms[index].image[j] = np.int32(line[j+5])

    return atoms,timestep


def lines_in_file(filename):
    """ Get the number of lines in the file """

    with open(filename) as f:
        for i, l in enumerate(f):
            pass

    return i + 1

In [3]:
def dbscan(atoms, threshold, target_type):
    if not atoms:
        return []

    cluster_id = 0
    cluster_ids = [-1] * len(atoms)  # Initialize cluster IDs for each atom; -1 means unclassified

    def find_neighbors(atom_index):
        return [i for i, other_atom in enumerate(atoms)
                if i != atom_index and atoms[atom_index].type == target_type and other_atom.type == target_type and 
                atoms[atom_index].sep(other_atom) < threshold]
    
    
    for i in range(len(atoms)):
        if cluster_ids[i] != -1 or atoms[i].type != target_type:
            if atoms[i].type != target_type:
                cluster_ids[i] = -2  # Mark atoms of non-interest with -2 or another special ID
            continue  # Skip if atom is already processed or not of target type

        neighbors = find_neighbors(i)

        cluster_id += 1
        cluster_ids[i] = cluster_id

        k = 0
        while k < len(neighbors):
            neighbor_idx = neighbors[k]
            if cluster_ids[neighbor_idx] == -1:
                cluster_ids[neighbor_idx] = cluster_id

                new_neighbors = find_neighbors(neighbor_idx)
                for new_neighbor in new_neighbors:
                    if cluster_ids[new_neighbor] == -1:
                        neighbors.append(new_neighbor)
            k += 1
            
    no_of_clusters = cluster_id # cluster_id acts like a 'cluster count'

    return no_of_clusters, cluster_ids


In [4]:
def size_of_clusters(cluster_ids):
    """ Returns a list of the number of proteins in a cluster """
    
    size_of_clusters = [0] * max(cluster_ids) # 0 means 0 atoms in cluster i
    
    # loop through list of cluster ids
    for i in cluster_ids:
        
        # if the cluster id is not -2 (-2 are atoms of non-interest)
        if i != -2:
            # increase value of i-1 by 1
            size_of_clusters[i-1] += 1
                    
    return size_of_clusters



def mean_size_of_clusters(size_of_clusters):
    """ Takes in list of size of clusters (in number of proteins) and returns the mean of proteins in a cluster """
    
    return s.fmean(size_of_clusters)



def size_of_largest_cluster(size_of_clusters):
    """ Takes in list of size of clusters and returns the no of proteins in largest cluster """
    
    return max(size_of_clusters)


def no_of_clusters_size_1(size_of_clusters):
    """ Takes in a list of cluster sizes and returns the number of clusters with only 1 protein - i.e. size 1 """

    return size_of_clusters.count(1)


In [5]:
def no_proteins_bound_to_poly(atoms, threshold=1.8):
    """ Takes in a list of Atom objects. 
        Returns the number of proteins bound to the polymer of type=1, 2 or 3 and a list of the number of polymer beads an atom is bound to.  """

    def find_polymers_bound_to(j):
        """ Takes in index of a protein (type 4) and finds if it is bound to polymer (type=1, 2 or 3). Bound to poly if within threshold distance of 1.8.
            Returns a list of indices of polymer beads of type=1, 2 or 3 for atom index inputted. 
            Length of list is no of polymer beads that protein is bound to. """

        return [i for i, other_atom in enumerate(atoms)
                if i != j and other_atom.type in {2} and atoms[j].sep(other_atom) < threshold]

    no_proteins_bound = 0 # intialises counter of number of proteins bound to a polymer bead to 0
    no_polymers_bound_to = [0] * len(atoms) # initialises a list of length of atoms to 0 - will contain the number of polymer beads atom ith is bound to

    # loops through all atoms
    for j, atom in enumerate(atoms):

        # if atom is a protein (type 4)
        if atom.type == 4:

            # find number of polymer beads (of any type - 1, 2 or 3) that protein is bound to
            poly_beads_list = find_polymers_bound_to(j)
            no_polymers_bound_to[j] = len(poly_beads_list) # length of poly_beads_list is the number of polymer beads that protein is bound to

            # if list is not empty
            if poly_beads_list:
                no_proteins_bound += 1

    return no_proteins_bound, no_polymers_bound_to # TO DO: TEST THIS!!!! ************

In [9]:
def fraction_clusters_bound_to_poly(atoms, cluster_ids, no_of_clusters, target_types={1, 2, 3}, threshold_2=3.24):
    """ Takes in a list of Atom objects and their cluster ids. 
        Returns the fraction of clusters bound to the polymer of type=1, 2 or 3 """

    def protein_bound_to_poly(j):
        """ Takes in index of a protein (type 4) and finds if it is bound to polymer (type=1, 2 or 3). 
            Bound to polymer if within threshold distance of 1.8. Or if sep_2 < threshold_2 (3.24). 
            Stops and returns True if it is bound. False if not bound. """
        for i, other_atom in enumerate(atoms):
            if i != j and other_atom.type in target_types and atoms[j].sep_2(other_atom) < threshold_2:
                return True # stop when you find a protein in cluster that is within threshold distance of any polymer bead
        return False

    bound_clusters = []  # list of cluster ids of bound clusters
    print("bound clusters list: ", bound_clusters)
    no_of_clusters1 = max(cluster_ids) # total no of clusters is the max value of cluster_ids list

    # loop through all cluster ids
    for j, cluster_id in enumerate(cluster_ids):
        
        # if cluster_id is not -2 (ie: it is the cluster id of a protein) and it is not already in bound_clusters
        if cluster_id != -2 and cluster_id not in bound_clusters:

            # check if the protein belonging to that cluster is bound to a polymer bead. If it is add to bound_clusters list.
            if protein_bound_to_poly(j):
                bound_clusters.append(cluster_id)

    fraction_bound = len(bound_clusters) / no_of_clusters # no of bound clusters / total no of clusters
    
    print("---------------------------------------------")
    print("input: ",no_of_clusters)    
    print("calc max : ",no_of_clusters1)
    print("bound clusters list: ", bound_clusters)
    print("no of bound clusters: ",len(bound_clusters))
    print("frac of bound clusters: ",fraction_bound)
    print("---------------------------------------------")

    return fraction_bound

In [12]:
### main programs --> need to change this cause very sucky!

######################## - User inputs (sys argv command line) but worse cause need to remember the order of inputs :/

"""
dumpfilename = sys.argv[1] # name of dump file to read - dump.sticky_DNA+proteins

Natoms = int(sys.argv[2]) # no of atoms - 220
Npoly = int(sys.argv[3]) # no of polymer atoms - 200

outfile_Rg = sys.argv[4] # name of output file - r_g_sticky_DNA

thresh = float(sys.argv[5]) # cluster threshold - 2.4
"""

######################## - Hardcoded (use for testing)

#name_dumpfile = 'dump.sticky_DNA+proteins' # name of dump file to read - dump.sticky_DNA+proteins

#n_atoms = 220 # no of atoms - 220
#n_poly_atoms = 200 # no of polymer atoms - 200

#name_outfile = 'r_g_sticky_DNA' # name of output file - r_g_sticky_DNA

######################## - User inputs (command line) - best way imo

name_dumpfile = input("Name of dumpfile: ")

n_atoms = int(input("Integer no of atoms (in df): "))
n_poly_atoms = int(input("Integer no of polymer atoms (in df): "))

name_outfile = input("Name of output file: ")

########################

# TO DO: could make this more automatic by giving a default of everything... user can say y/n to default and change if required

threshold = 2.4 # cluster threshold - 2.4
target_type = 2  # target atom type - should change this to type 4 for model simulations... / could add as user input

path_to_dumpfiles = '../../lammps_sims/dumpfiles/' # need to change this when running on local computer
path_to_outfiles = '../outfiles/'

n_lines = lines_in_file(path_to_dumpfiles + name_dumpfile) # no of lines in file
n_frames = int(n_lines / (n_atoms + 9)) # +9 as 9 header lines in each frame

# open the input file
file_in = open(path_to_dumpfiles + name_dumpfile, 'r')

# open the output file and print a header
file_out = open(path_to_outfiles + name_outfile, 'w')
file_out.write("# Timesteps, No of clusters, Fraction of clusters bound\n")

# go through the file frame by frame
for frame in range(n_frames):
    # read the frame, unwrapping periodic coordinates
    atoms, timesteps = readframe(file_in, n_atoms)
    
    # unwarp period boundary coordinates -- needed for clusters? - think so as a cluster can form between boundaries
    for i in range(len(atoms)):
        atoms[i].unwrap()

    # perform calculations
    no_of_clusters, cluster_ids = dbscan(atoms, threshold, target_type)
    #cluster_size = size_of_clusters(cluster_ids)
    #mean_cluster_size = mean_size_of_clusters(cluster_size)
    #largest_cluster_size = size_of_largest_cluster(cluster_size)
    
    #no_proteins_bound, no_polymers_bound_to = no_proteins_bound_to_poly(atoms)
    
    frac_clusters_bound = fraction_clusters_bound_to_poly(atoms, cluster_ids, no_of_clusters)
    
    # output some results
    #file_out.write("%i %i %i %i\n"%(timesteps, no_of_clusters, mean_cluster_size, largest_cluster_size))
    file_out.write("%i %i %.2f\n"%(timesteps, no_of_clusters, frac_clusters_bound))

# close the files
file_in.close()
file_out.close()

Name of dumpfile: dump_model_4_var_8_run_1.dat
Integer no of atoms (in df): 5300
Integer no of polymer atoms (in df): 5000
Name of output file: test_outfile.dat
bound clusters list:  []
---------------------------------------------
input:  76
calc max :  76
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]
no of bound clusters:  76
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  65
calc max :  65
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 

---------------------------------------------
input:  38
calc max :  38
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
no of bound clusters:  38
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  37
calc max :  37
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
no of bound clusters:  37
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  35
calc max :  35
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
no of bound clusters:  35
frac of bound clusters: 

bound clusters list:  []
---------------------------------------------
input:  27
calc max :  27
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
no of bound clusters:  27
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  27
calc max :  27
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
no of bound clusters:  27
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  27
calc max :  27
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
no of bound clusters:  27
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------

bound clusters list:  []
---------------------------------------------
input:  30
calc max :  30
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
no of bound clusters:  30
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  28
calc max :  28
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
no of bound clusters:  28
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  27
calc max :  27
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
no of bound clusters:  27
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []

bound clusters list:  []
---------------------------------------------
input:  21
calc max :  21
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
no of bound clusters:  21
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  23
calc max :  23
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
no of bound clusters:  23
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  20
calc max :  20
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
no of bound clusters:  20
frac of bound clusters:  1.0
---------------------------------------------
bound clusters list:  []
---------------------------------------------
input:  22
calc max :  22
bound clust

KeyboardInterrupt: 

In [8]:
# took ~3 mins to run above - note that it was reduced to just checking for type 2 polymer atoms!